### Set Up Env

Process Flow (Small Test Site)
- In first instance get Coventry OAs
- Select small area of city using bounding box and identify intersecting OAs - use this as area of study
- Get POIs - lets use schools
- Get underlying OSM data and output as osm file

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, box, Point
import r5py

rating_mapping = {'Outstanding': 1,
                  'Good': 3,
                  'Requires improvement':5}

In [2]:
bounding_box = tuple([-1.925354,52.437594,-1.835747,52.491145])

In [3]:
#Get OAs is area of study
oas = gpd.read_file('data/OAs/OA_2021_EW_BGC.shp').to_crs(4326)
bbox = box(bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])
gpd.GeoSeries([bbox]).to_file('data/outputs/area_of_study.geojson', driver='GeoJSON')

ids = []
origin_centroids = []
origin_points = []
count = 0
count_found = 0
for i,r in oas.iterrows():
    count += 1
    if count % 100 == 0:
        print(count / len(oas))
    if r['geometry'].intersects(bbox):
        count_found += 1
        ids.append(r['OA21CD'])
        origin_centroids.append([r['geometry'].centroid.x,r['geometry'].centroid.y])
        origin_points.append(Point([r['geometry'].centroid.x,r['geometry'].centroid.y]))
        
print('Number found : {}'.format(count_found))
oas_in_study = oas[oas['OA21CD'].isin(ids)]
oas_in_study = oas_in_study.set_index('OA21CD')

0.0005294366793731469
0.0010588733587462938
0.0015883100381194409
0.0021177467174925877
0.0026471833968657347
0.0031766200762388818
0.0037060567556120288
0.004235493434985175
0.004764930114358323
0.0052943667937314694
0.005823803473104617
0.0063532401524777635
0.006882676831850911
0.0074121135112240576
0.007941550190597205
0.00847098686997035
0.009000423549343498
0.009529860228716646
0.010059296908089793
0.010588733587462939
0.011118170266836086
0.011647606946209234
0.01217704362558238
0.012706480304955527
0.013235916984328674
0.013765353663701822
0.014294790343074968
0.014824227022448115
0.015353663701821263
0.01588310038119441
0.016412537060567556
0.0169419737399407
0.01747141041931385
0.018000847098686996
0.018530283778060146
0.01905972045743329
0.019589157136806437
0.020118593816179586
0.020648030495552732
0.021177467174925878
0.021706903854299027
0.022236340533672173
0.02276577721304532
0.023295213892418468
0.023824650571791613
0.02435408725116476
0.02488352393053791
0.02541296060

In [4]:
oas_in_study['id'] = oas_in_study['GlobalID']
oas_in_study['geometry'] = origin_points

In [5]:
#Govt School Data
schools_birm = pd.read_csv('data/POIs/birm_schools.csv')

#Filter primary schools publicly funded
schools_birm = schools_birm[schools_birm['CLOSEDATE'].isna()]
schools_birm = schools_birm[schools_birm['ISPRIMARY'] == 1]
schools_birm = schools_birm[schools_birm['MINORGROUP'].isin(['Maintained school','Academy'])]
schools_birm = schools_birm[schools_birm['OFSTEDRATING'].isin(['Good','Outstanding','Requires improvement'])]
schools_birm = schools_birm[schools_birm['LANAME'] == 'Birmingham']

In [6]:
school_points = []
destination_centroids = []
attractiveness = []
for i,r in schools_birm.iterrows():
    school_points.append(Point([r['Longitude'],r['Latitude']]))
    destination_centroids.append([r['Longitude'],r['Latitude']])
    attractiveness.append(rating_mapping[r['OFSTEDRATING']])
    
schools_birm['geometry'] = school_points
schools_birm['attractiveness'] = attractiveness
schools_birm['id'] = schools_birm['URN']

# create a geodataframe in a specific CRS and save the result into a GeoJson file
poi_gdf = gpd.GeoDataFrame(schools_birm[['geometry','attractiveness','id']], crs="EPSG:4326")

poi_gdf.set_geometry('geometry', crs={'init': 'epsg:4326'})
poi_gdf.to_file('data/outputs/POI_schools_govt.geojson', driver='GeoJSON')

/home/chris/miniconda3/envs/access/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
# Get Start Times
# Select random start time between 9am and 6pm on 10th May 2022

In [7]:
from datetime import datetime, timedelta
import random

# Set the date
date_to_generate = datetime(2022, 5, 10)

# Set the start and end times
start_time = datetime.strptime('09:00', '%H:%M')
end_time = datetime.strptime('17:00', '%H:%M')

# Generate 100 random times
random_times = [start_time + timedelta(minutes=random.randint(0, (end_time - start_time).seconds // 60)) for _ in range(100)]

# Apply the generated times to the date
random_datetimes = [datetime(date_to_generate.year, date_to_generate.month, date_to_generate.day,
                             time.hour, time.minute) for time in random_times]

In [10]:
random_datetimes[0]

datetime.datetime(2022, 5, 10, 16, 17)

In [ ]:
#Test routing using r5 from each OA to each school

In [8]:
transport_network = r5py.TransportNetwork(
    'data/west-midlands-latest.osm.pbf',
    ['data/tfwm_gtfs.zip'],
)

,GlobalID,geometry,id
OA21CD,,,
E00045131,d07bc9f4-175c-45d7-bf9c-395a194872aa,POINT (-1.84034 52.46142),d07bc9f4-175c-45d7-bf9c-395a194872aa
E00045134,61b80771-5d10-4a37-8e0b-0f5546b9e8e2,POINT (-1.83378 52.45995),61b80771-5d10-4a37-8e0b-0f5546b9e8e2
E00045135,d0ff2cc4-84da-4691-8c60-0b74acadf0e8,POINT (-1.83485 52.46225),d0ff2cc4-84da-4691-8c60-0b74acadf0e8
E00045162,b91e20dc-a9e0-45e9-93c5-23946d07aa13,POINT (-1.83953 52.45600),b91e20dc-a9e0-45e9-93c5-23946d07aa13
E00045206,2e1e9655-a5f9-4235-af91-1cf114d5f24f,POINT (-1.90783 52.49266),2e1e9655-a5f9-4235-af91-1cf114d5f24f


,geometry,attractiveness,id
0,POINT (-1.84586 52.48721),1,103157
1,POINT (-1.86104 52.48382),3,103159
2,POINT (-1.93465 52.48413),3,103162
3,POINT (-1.90783 52.54061),3,103163
4,POINT (-1.90783 52.54061),3,103164


In [20]:
travel_time_matrix_computer = r5py.DetailedItinerariesComputer(
    transport_network,
    origins=oas_in_study[:1],
    destinations=poi_gdf[:1],
    departure=random_datetimes[0],
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
    snap_to_network=True,
)

In [18]:
travel_times = travel_time_matrix_computer.compute_travel_details()

/home/chris/miniconda3/envs/access/lib/python3.12/site-packages/r5py/r5/detailed_itineraries_computer.py:135: RuntimeWarning: R5 has been compiled with `TransitLayer.SAVE_SHAPES = false` (the default). The geometries of public transport routes are inaccurate (straight lines between stops), and distances can not be computed.
  warnings.warn(


In [19]:
travel_times

,from_id,to_id,option,segment,transport_mode,departure_time,distance,travel_time,wait_time,route,geometry
0,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,0,0,TransportMode.WALK,NaT,3734.390,0 days 01:02:40,NaT,None,"LINESTRING (-1.84024 52.46141, -1.84081 52.461..."
1,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,1,0,TransportMode.WALK,2022-05-10 15:42:27,593.714,0 days 00:10:02,0 days 00:00:00,None,"LINESTRING (-1.84024 52.46141, -1.84081 52.461..."
2,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,1,1,TransportMode.BUS,2022-05-10 15:58:00,NaN,0 days 00:10:00,0 days 00:05:43,41,"LINESTRING (-1.83686 52.46103, -1.83275 52.464..."
3,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,1,2,TransportMode.WALK,2022-05-10 16:09:00,172.877,0 days 00:02:56,0 days 00:01:02,None,"LINESTRING (-1.83103 52.48029, -1.83167 52.480..."
4,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,1,3,TransportMode.BUS,2022-05-10 16:12:19,NaN,0 days 00:03:46,0 days 00:01:21,28,"LINESTRING (-1.83295 52.48068, -1.83167 52.483..."
...,...,...,...,...,...,...,...,...,...,...,...
416,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,60,2,TransportMode.WALK,2022-05-10 16:09:00,172.877,0 days 00:02:56,0 days 00:01:02,None,"LINESTRING (-1.83103 52.48029, -1.83167 52.480..."
417,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,60,3,TransportMode.BUS,2022-05-10 16:12:19,NaN,0 days 00:03:46,0 days 00:01:21,28,"LINESTRING (-1.83295 52.48068, -1.83167 52.483..."
418,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,60,4,TransportMode.WALK,2022-05-10 16:17:05,219.771,0 days 00:03:39,0 days 00:01:08,None,"LINESTRING (-1.83070 52.48613, -1.83067 52.486..."
419,d07bc9f4-175c-45d7-bf9c-395a194872aa,103157,60,5,TransportMode.BUS,2022-05-10 16:21:59,NaN,0 days 00:05:33,0 days 00:02:07,14,"LINESTRING (-1.83182 52.48737, -1.83592 52.486..."
